In [19]:
import pandas as pd
import folium
import numpy as np
import os

In [2]:
df = pd.read_csv('Untitled spreadsheet - Sheet1.csv')

In [5]:
df = df.drop(columns="Unnamed: 13")

In [10]:
df['Long/Lant'].str.split?

In [16]:
df[['lon','lat']] = df['Long/Lant'].str.split(',',expand=True).astype(float)

In [22]:
def popup_html(row_index):
    # Initialize the HTML content with a <style> tag to change the font
    html_content = '''
    <style>
        .popup-content {
            font-family: Arial, sans-serif; /* Change the font family as needed */
            font-size: 14px;
        }
        img {
            max-width: 100%; /* Ensure images are not too wide */
            height: auto; /* Maintain aspect ratio */
            margin-top: 5px; /* Add some space above the image */
        }
    </style>
    <div class="popup-content">
    '''

    # Dynamically add data for each column that has a value
    for column in df.columns:
        value = df.at[row_index, column]
        value_str = str(value).strip()
        if pd.notnull(value) and value_str != "":
            if column == 'image_links' and value_str != "No Image":  # Assuming 'No Image' indicates absence of an image
                # Embed the image using an <img> tag
                html_content += f'<img src="{value_str}" alt="Image"><br>'
            elif column not in ['lat', 'lon', 'Long/Lant', 'image_links']:  # Exclude certain columns from the text part
                html_content += f'<strong>{column}:</strong> {value_str}<br>'

    # Close the div tag
    html_content += '</div>'

    return html_content

# Define your map 'm' here as needed
m = folium.Map(location=df[["lon", "lat"]].mean().to_list(), zoom_start=16)

# Example of how to use the popup_html function within the loop for adding markers
for i in range(len(df)):
    popup_content = popup_html(i)
    iframe = folium.IFrame(html=popup_content, width=300, height=300)  # Adjust iframe size as needed
    popup = folium.Popup(iframe, max_width=350)  # Adjust max_width as needed
    folium.Marker(
        location=[df.iloc[i]['lon'], df.iloc[i]['lat']],
        popup=popup
    ).add_to(m)
    
m

In [23]:
m.save('index.html')